## Import der benötigten Bibliotheken

In [ ]:
import pprint
import csv
import pandas as pd
from pymongo import MongoClient

## Konvertierung der .txt-Datenquellen in .csv-Dateien
Muss nur beim ersten Mal ausgeführt werden

In [ ]:
sentences_csv = pd.read_csv ('deu_news_2015_3M-sentences.txt', header = None, delimiter='\t')
sentences_csv.to_csv ('sentences_de.csv', index=None)

In [ ]:
stopwords_csv = pd.read_csv ('stopwords_deutsch.txt', header = None, delimiter='\t')
stopwords_csv.to_csv ('stopwords_de.csv', index=None)

## Verbindung mit mongoDB

In [ ]:
connection_string = "mongodb://localhost:27017"
db                = MongoClient(connection_string)
sentences         = db.dw.sentences
stopwords         = db.dw.stopwords

## Sentences und Stopwords der Database in mongoDB hinzufügen

`read_csv()` liest eine .csv-Datei ein und gibt ein Dictionary zurück, dass dann mit `save_to_mongo()` in einer Collection gespeichert wird. Dafür kann zunächst die Collection gelöscht werden.

In [ ]:
def read_csv(path, columns):
    reader = open(path, 'r', encoding='utf-8')
    return csv.DictReader(reader, columns)   

In [ ]:
def save_to_mongo(collection, path, columns, drop_collection):
    if drop_collection:
        collection.drop()
    data = read_csv(path, columns)
    result = collection.insert_many(data)
    print('%d rows are saved to "%s" collection successfully!' % (len(result.inserted_ids), collection.name))

In [ ]:
save_to_mongo(sentences, 'sentences_de.csv', ('id','sentence'), True)

In [ ]:
save_to_mongo(stopwords, 'stopwords_de.csv', ('s'), True)

Wenn ich die als Column-Name gleich `stopwords` genommen habe, dann hat er daraus eine Spalte `s`, eine `t`, eine `o` und so weiter gemacht...Deswegen hier mit der späteren Umbenennung.

In [ ]:
stopwords.update_many( {}, { '$rename': { "s": "stopword" } } )